In [1]:
import random
import numpy as np
import math
import time
import os

In [2]:
def euclidean_distance(a, b):
    return math.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

In [3]:
def genetic_backpack(max_capacity, w, c):
    N = len(w)
    n_chromo = 4
    dna = np.zeros(N*n_chromo).reshape(n_chromo, N)
    k = 2
    # генерируем хромосомы
    for i in range(n_chromo):
        for j in range(N):
            dna[i][j] = random.randint(0,1)
    #print('Популяция:\n', dna)
    def remove_random_item(chromo):
        one_positions = []
        for i in range(len(chromo)):
            if chromo[i]:
                one_positions.append(i)
        chromo[one_positions[random.randint(0, len(one_positions) - 1)]] = 0
        
    # Считаем вес и стоимость
    def fitness_function(chromo, w, c):
        temp_w, temp_p = 0, 0
        for j in range(N):
            if chromo[j]:
                temp_w += w[j]
                temp_p += c[j]
        return temp_p, temp_w
    total_chromo = []
    for i in range(n_chromo):
        temp_p, temp_w = fitness_function(dna[i], w, c)
        
        while temp_w > max_capacity:
            remove_random_item(dna[i])
            temp_p, temp_w = fitness_function(dna[i], w, c)
        total_chromo.append((i, temp_p, temp_w))
    #print('Популяция после подсчета фит. функции:\n', dna)  
    
    # Сортируем
    total_chromo = sorted(total_chromo, key=lambda x: x[1], reverse=True)
    print(total_chromo)
    
    # Находим распределение вероятностей
    fitness_sum = 0
    for x in total_chromo:
        fitness_sum += x[1]
    distribution = []
    for x in total_chromo:
        distribution.append(x[1] / fitness_sum)
    #distribution.sort(reverse=True)
    
    # выбираем для селекции
    chromos_selected = random.choices(dna, weights=distribution, k=k)
    # чтобы не брать одинаковые
    while len(np.unique(chromos_selected, axis=0)) < k:
        chromos_selected = random.choices(dna, weights=distribution, k=k)
    #print('Выбрали для селекции:\n',chromos_selected)
    
    # делаем селекцию
    crossover_point = random.randint(1, N-1)
    tmp = chromos_selected[1][:crossover_point].copy()
    chromos_selected[1][:crossover_point], chromos_selected[0][:crossover_point] = chromos_selected[0][:crossover_point], tmp
    #print('После селекции:\n', chromos_selected)
    
    # мутация
    for j in range(len(chromos_selected)):
        for i in range(len(chromos_selected[j])):
            if random.randint(1, 100) == 69:
                chromos_selected[j][i] = (chromos_selected[j][i] + 1) % 2
    #print('После мутации:\n', chromos_selected)
    
    # считаем ф-ф полученных хромо, выравниваем веса
    fitness_results_selected = []
    for i in range(len(chromos_selected)):
        temp_p, temp_w = fitness_function(chromos_selected[i], w, c)
        while temp_w > max_capacity:
            remove_random_item(chromos_selected[i])
            temp_p, temp_w = fitness_function(chromos_selected[i], w, c)
        fitness_results_selected.append((i, temp_p, temp_w))

    fitness_results_selected.sort(key=lambda x: x[1], reverse=True)

    if fitness_results_selected[0][1] > total_chromo[0][1]:
        return chromos_selected[fitness_results_selected[0][0]], fitness_results_selected[0][1], fitness_results_selected[0][2]
    else:
        return dna[total_chromo[0][0]], total_chromo[0][1], total_chromo[0][2]
    return

In [108]:
W = [23, 31, 29, 44, 53, 38, 63, 85, 89, 82]
C = [92, 57, 49, 68, 60, 43, 67, 84, 87, 72]
#items, cap, profit = genetic_backpack(10, 165, W, C)
genetic_backpack(10, 165, W, C)

Популяция:
 [[0. 0. 0. 1. 1. 0. 1. 1. 0. 0.]
 [0. 1. 0. 0. 1. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 0. 1. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1. 0. 0. 1. 0. 1.]]
Популяция после подсчета фит. функции:
 [[0. 0. 0. 1. 1. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]]
[(2, 197, 157), (0, 195, 160), (3, 128, 97), (1, 110, 101)]
Выбрали для селекции:
 [array([0., 1., 0., 1., 0., 0., 0., 0., 0., 1.]), array([0., 0., 0., 0., 0., 1., 1., 0., 0., 0.])]
После селекции:
 [array([0., 0., 0., 1., 0., 0., 0., 0., 0., 1.]), array([0., 1., 0., 0., 0., 1., 1., 0., 0., 0.])]
После мутации:
 [array([0., 0., 0., 1., 0., 0., 0., 0., 0., 1.]), array([0., 1., 0., 0., 0., 1., 1., 0., 0., 0.])]


(array([0., 0., 0., 1., 0., 0., 0., 0., 0., 1.]), 197, 157)

In [11]:
p_c = []
p_p = []
p_s = []
p_w = []
for i in range(1, 8):
    p_c.append(open(os.getcwd() + "\\benchmarks_knapsack\\p0" + str(i) + "_c.txt", "r"))
    p_p.append(open(os.getcwd() + "\\benchmarks_knapsack\\p0" + str(i) + "_p.txt", "r"))
    p_s.append(open(os.getcwd() + "\\benchmarks_knapsack\\p0" + str(i) + "_s.txt", "r"))
    p_w.append(open(os.getcwd() + "\\benchmarks_knapsack\\p0" + str(i) + "_w.txt", "r"))
for test_num in range(7):
    print(f"BENCHMARK p0{test_num+1}")
    w = int(p_c[test_num].read())
    items_c = []
    for line in p_p[test_num]:
        items_c.append(int(line))
        
    items_w = []
    for line in p_w[test_num]:
         items_w.append(int(line))

    items_true = []
    w_true = 0
    c_true = 0
    for line in p_s[test_num]:
        items_true.append(int(line))
    for j in range(len(items_true)):
        if items_true[j] == 1:         
            w_true += items_w[j]
            c_true += items_c[j]
                
    answers = []
    timeExecuted = 0
    for i in range(100):
        start = time.time()
        answers.append(genetic_backpack(w, items_w, items_c))
        timeExecuted += time.time() - start
    print(f"TIME: {timeExecuted/100*1000}")
    bestResult = max(list(map(lambda x: x[1], answers)))
    print(f'COST: {bestResult}')
    for i in answers:
        if i[1] == bestResult:
            print(f'ITEMS: {i[0]}')
            break

BENCHMARK p01
[(2, 204, 162), (0, 136, 118), (3, 128, 97), (1, 117, 84)]
[(3, 260, 136), (1, 219, 139), (0, 159, 86), (2, 155, 133)]
[(2, 270, 165), (0, 213, 134), (3, 159, 130), (1, 140, 126)]
[(1, 224, 158), (3, 164, 105), (2, 139, 145), (0, 127, 123)]
[(0, 241, 121), (2, 239, 165), (1, 184, 136), (3, 183, 164)]
[(1, 259, 155), (3, 209, 160), (0, 160, 111), (2, 121, 111)]
[(1, 258, 136), (3, 209, 160), (0, 159, 86), (2, 159, 86)]
[(3, 221, 136), (1, 155, 133), (0, 151, 148), (2, 84, 85)]
[(0, 232, 149), (2, 179, 112), (3, 87, 89), (1, 72, 82)]
[(0, 309, 165), (3, 184, 147), (2, 147, 142), (1, 144, 138)]
[(0, 178, 145), (2, 164, 105), (3, 160, 122), (1, 140, 126)]
[(1, 225, 137), (2, 209, 160), (3, 207, 143), (0, 154, 152)]
[(1, 213, 134), (2, 207, 143), (3, 130, 127), (0, 129, 113)]
[(1, 198, 83), (3, 147, 142), (0, 87, 89), (2, 0, 0)]
[(0, 270, 165), (3, 219, 146), (2, 84, 85), (1, 72, 82)]
[(0, 208, 115), (2, 184, 136), (3, 178, 142), (1, 151, 148)]
[(2, 252, 145), (3, 220, 120), (

[(2, 1322, 684), (0, 1296, 673), (3, 1011, 529), (1, 981, 505)]
[(2, 1369, 712), (0, 1306, 680), (3, 1251, 653), (1, 1060, 558)]
[(0, 1436, 749), (2, 1314, 683), (3, 1221, 635), (1, 957, 498)]
[(3, 1434, 743), (2, 1267, 658), (1, 901, 468), (0, 686, 360)]
[(0, 1332, 688), (1, 1257, 653), (3, 742, 385), (2, 477, 248)]
[(2, 1312, 678), (0, 1218, 638), (1, 1178, 608), (3, 1119, 580)]
[(2, 1423, 745), (3, 1331, 689), (0, 1287, 677), (1, 690, 359)]
[(3, 1402, 735), (2, 1369, 715), (1, 1171, 611), (0, 941, 487)]
[(1, 1340, 702), (2, 1232, 648), (3, 1189, 621), (0, 961, 496)]
[(2, 1431, 747), (0, 1230, 638), (3, 1227, 644), (1, 510, 268)]
[(1, 1361, 707), (0, 1304, 680), (3, 1291, 668), (2, 1275, 669)]
[(0, 1328, 688), (2, 1280, 668), (3, 1277, 667), (1, 811, 427)]
[(0, 1395, 728), (1, 1092, 569), (2, 990, 514), (3, 816, 428)]
[(0, 1440, 750), (3, 1336, 698), (2, 1332, 692), (1, 1318, 686)]
[(3, 1446, 747), (0, 1340, 698), (1, 1303, 675), (2, 1087, 566)]
[(0, 1330, 693), (2, 1225, 639), (1, 9

In [4]:
def fitness_function(chromosome, edges_weight):
    ans = 0
    for i in range(len(chromosome)-1):
        ans += edges_weight[chromosome[i]][chromosome[i+1]]
    return ans


def get_ans(edges_weight):
    num_of_edges = len(edges_weight)
    num_of_chromosomes = 1000
    k = num_of_chromosomes // 4
    k = k // 4
    k *= 4
    chromosomes = []
    for i in range(num_of_chromosomes):
        chromosomes.append([])
        for j in range(num_of_edges):
            chromosomes[i].append(j)
        random.shuffle(chromosomes[i])

    fitness_results = []
    for i in range(num_of_chromosomes):
        tmp = fitness_function(chromosomes[i], edges_weight)
        fitness_results.append((i, tmp))
    fitness_results.sort(key=lambda x: x[1])

    random_list = [0 for i in range(num_of_chromosomes)]
    for i in range(k):
        random_list[i] = 1
    random.shuffle(random_list)

    chromosomes_selected = []
    for i in range(num_of_chromosomes):
        if random_list[i] == 1:
            chromosomes_selected.append((i, fitness_results[i]))

    chromosomes_selected.sort(key=lambda x: x[1])
    chromosomes_selected = chromosomes_selected[:len(chromosomes_selected)//2]

    for i in range(0, len(chromosomes_selected), 2):

        crossover_point1 = random.randint(0, num_of_edges - 2)

        crossover_point2 = random.randint(crossover_point1+1, num_of_edges)

        child1 = [-1 for i in range(num_of_edges)]

        child2 = [-1 for i in range(num_of_edges)]

        tmp1 = chromosomes[chromosomes_selected[i+1][0]][crossover_point2:] + chromosomes[chromosomes_selected[i+1][0]][:crossover_point1]
        tmp2 = chromosomes[chromosomes_selected[i][0]][crossover_point2:] + chromosomes[chromosomes_selected[i][0]][:crossover_point1]

        for j in range(crossover_point1, crossover_point2):
            child1[j] = chromosomes[chromosomes_selected[i][0]][j]
            child2[j] = chromosomes[chromosomes_selected[i+1][0]][j]
            if chromosomes[chromosomes_selected[i+1][0]][j] in tmp1:
                tmp1.remove(chromosomes[chromosomes_selected[i+1][0]][j])
            if chromosomes[chromosomes_selected[i][0]][j] in tmp2:
                tmp2.remove(chromosomes[chromosomes_selected[i][0]][j])

        child1[crossover_point2:] = tmp1[:num_of_edges - crossover_point2]
        child1[:crossover_point1] = tmp1[num_of_edges - crossover_point2:]

        child2[crossover_point2:] = tmp2[:num_of_edges - crossover_point2]
        child2[:crossover_point1] = tmp2[num_of_edges - crossover_point2:]

        if random.randint(1, 100) == 42:
            pointer1 = random.randint(0, num_of_edges - 1)
            pointer2 = random.randint(0, num_of_edges - 1)
            child1[pointer1], child1[pointer2] = child1[pointer2], child1[pointer1]

        if random.randint(1, 100) == 42:
            pointer1 = random.randint(0, num_of_edges - 1)
            pointer2 = random.randint(0, num_of_edges - 1)
            child2[pointer1], child2[pointer2] = child2[pointer2], child2[pointer1]

        chromosomes.append(child1)
        chromosomes.append(child2)

    for i in range(num_of_chromosomes, len(chromosomes)):
        tmp = fitness_function(chromosomes[i], edges_weight)
        fitness_results.append((i, tmp))
    fitness_results.sort(key=lambda x: x[1])

    return fitness_results[0][1], chromosomes[fitness_results[0][0]]

In [12]:
salesman_benchmarks = {'a280': open('benchmarks_salesman/a280.tsp', 'r').readlines()[6:-1],
                           'att48': open('benchmarks_salesman/att48.tsp', 'r').readlines()[6:-1],
                           'bays29': open('benchmarks_salesman/bays29.tsp', 'r').readlines()[38:-1],
                           'ch150': open('benchmarks_salesman/ch150.tsp', 'r').readlines()[6:-1],
                           'fl417': open('benchmarks_salesman/fl417.tsp', 'r').readlines()[6:-1],
                           }

for benchmark in salesman_benchmarks:
    coord = []
    for line in salesman_benchmarks[benchmark]:
        coord.append((float(line.split()[1]), float(line.split()[2])))
    w = []
    for i in range(len(coord)):
        w.append([])
        for j in range(len(coord)):
            w[i].append(euclidean_distance(coord[i], coord[j]))
    answers = []
    timeExecuted = 0
    for i in range(100):
        start = time.time()
        answer = get_ans(w)
        timeExecuted += time.time() - start
        answers.append(answer)

    print(f"BENCHMARK {benchmark}")
    print(f"TIME {timeExecuted/100*1000}")
    bestResult = min(list(map(lambda x: x[1], answers)))
    print(f'BEST RESULT: {bestResult}')
    for i in answers:
        if i[1] == bestResult:
            print(f'PATH: {i[0]}')
            break

BENCHMARK a280
TIME 168.8417625427246
BEST RESULT: [5, 133, 162, 22, 169, 67, 134, 181, 75, 62, 56, 50, 271, 220, 244, 73, 115, 173, 11, 9, 125, 132, 242, 144, 147, 137, 124, 179, 84, 47, 26, 98, 0, 240, 268, 76, 275, 270, 262, 83, 30, 266, 230, 74, 118, 141, 63, 78, 149, 79, 221, 251, 27, 104, 34, 23, 36, 29, 45, 81, 253, 194, 42, 97, 180, 157, 101, 191, 123, 121, 12, 256, 229, 40, 246, 198, 114, 235, 139, 28, 105, 53, 91, 213, 197, 188, 185, 161, 272, 274, 261, 200, 68, 215, 174, 66, 167, 182, 249, 172, 189, 217, 135, 146, 92, 168, 54, 170, 190, 163, 152, 279, 3, 278, 102, 46, 94, 228, 4, 267, 216, 2, 178, 193, 140, 65, 131, 82, 72, 129, 60, 184, 64, 223, 59, 171, 155, 241, 250, 87, 166, 148, 32, 204, 192, 164, 259, 55, 111, 96, 236, 106, 263, 248, 10, 16, 211, 150, 69, 80, 103, 222, 234, 238, 61, 127, 21, 214, 130, 116, 117, 258, 219, 245, 254, 224, 183, 77, 226, 31, 24, 89, 151, 13, 43, 57, 208, 153, 51, 119, 177, 252, 165, 107, 158, 70, 108, 112, 95, 218, 210, 237, 44, 25, 175, 7,